In [ ]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px

# Scoring Parameters/Times

In [ ]:
info = subs.a13_info
info['paths']['laser1']

# load a second of the data to get the metadata
data = tdt.read_block(info["paths"]['laser1'], t1=0, t2=1)

# and load all of the epocs
ep = tdt.read_block(info["paths"]['laser1'], t1=0, t2=0, evtype=['epocs'])

#start of the file
start_time = np.datetime64(data.info.start_date)

# First times we want to know are the start and end times of the laser stimulation:
stim_on = ep.epocs.Wdr_.onset[2]
stim_off = ep.epocs.Wdr_.offset[2]

# Then we will give ourselves a 5-min buffer on either side of the laser stimulation
stim_on = stim_on - 5*60
stim_off = stim_off + 5*60

# Then we also want to set the times for the second bout of receovery sleep:
# First we will specify RS2 as starting at 2:05pm, which is ~a 5min buffer from when I noted it as ending
rs_start = np.datetime64('2022-09-09T14:05:00')
# Then we get that as a TDT-time
rs_start_sec = (rs_start - start_time) / np.timedelta64(1, 's')


In [ ]:
a13_info = subs.a13_info
data, spg, bp, hyp = acr.io.acr_load_master(a13_info, type="pandas", stores=["EEGr", "LFP_", "LFPo"])

In [ ]:
acr.io.save_dataframes(hyp, a13_info, "-hypno")
acr.io.save_dataframes(bp, a13_info, "-bp")
hyp = acr.io.load_saved_dataset(a13_info, None, '-hypno')

In [ ]:
bp = acr.io.load_saved_dataset(a13_info, ['-LFP_', '-LFPo'], '-bp')

In [ ]:
# get the average delta value for each channel
bp = kpd.pd_utils.add_states_to_dataset(bp, hyp)

In [ ]:
pm = bp['laser1-bl-LFP_'].filt_state(['NREM']).groupby('channel').mean()

In [ ]:
om = bp['laser1-bl-LFPo'].filt_state(['NREM']).groupby('channel').mean()

In [ ]:
p = bp['laser1-LFP_']
o = bp['laser1-LFPo']

In [ ]:
stim_p = p.ts(slice(np.datetime64('2022-09-09T10:05:00'), np.datetime64('2022-09-09T12:10:00')))

In [ ]:
stim_o = o.ts(slice(np.datetime64('2022-09-09T10:05:00'), np.datetime64('2022-09-09T12:10:00')))

In [ ]:
stim_p.filt_state()

In [ ]:
bp['laser1-bl-LFPo'].filt_state(['NREM']).ch(4).delta.median()

In [ ]:
bp['laser1-LFPo'].filt_state(['NREM']).ch(4).delta.median()

In [ ]:
p.reset_index(inplace=True)

In [ ]:
op.groupby(['channel', ).mean().plot()

In [ ]:
stim_p.set_index(['datetime','channel'], inplace=True)

In [ ]:
stim_o.set_index(['datetime','channel'], inplace=True)

In [187]:
stim_p['delta'] = stim_p['delta']/pm['delta']

In [188]:
stim_o['delta'] = stim_o['delta']/om['delta']

In [189]:
op_stim = pd.concat([stim_p, stim_o])

In [193]:
op_stim.filt_state()

,datetime,channel,time,delta,delta1,delta2,theta,alpha,sigma,beta,gamma,timedelta,condition,state,dtype
0,2022-09-09 11:38:46.069451204,1,16353.069452,1.511103,3297.125000,40449.015625,11178.346680,3833.282471,4243.965820,8980.291992,5095.917480,0 days 00:04:44.069068568,laser1-LFP_,NREM,Probe
1,2022-09-09 11:38:46.069451204,2,16353.069452,1.586578,2638.751953,27210.921875,8570.375977,2794.405273,2863.622803,6383.524414,3295.919189,0 days 00:04:44.069068568,laser1-LFP_,NREM,Probe
2,2022-09-09 11:38:46.069451204,3,16353.069452,1.678088,3381.756592,20599.492188,7603.391602,2155.417236,2799.084961,5776.173340,2952.935547,0 days 00:04:44.069068568,laser1-LFP_,NREM,Probe
3,2022-09-09 11:38:46.069451204,4,16353.069452,1.915104,4107.337891,20216.179688,9473.392578,2642.876953,2957.202637,6352.410156,3543.478271,0 days 00:04:44.069068568,laser1-LFP_,NREM,Probe
4,2022-09-09 11:38:46.069451204,5,16353.069452,1.725349,4918.340820,19504.968750,9550.752930,2967.222656,2739.776123,5721.365723,3756.514404,0 days 00:04:44.069068568,laser1-LFP_,NREM,Probe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23707,2022-09-09 12:07:10.483862607,12,18057.483864,0.675432,8244.575195,2122.593018,20599.644531,1455.172974,1620.978027,3103.030518,1230.149658,0 days 00:33:08.483479971,laser1-LFPo,NREM,Optrode
23708,2022-09-09 12:07:10.483862607,13,18057.483864,0.342497,6882.299805,1710.145264,30160.775391,2029.403320,1474.898315,2855.687500,1338.048706,0 days 00:33:08.483479971,laser1-LFPo,NREM,Optrode
23709,2022-09-09 12:07:10.483862607,14,18057.483864,0.330020,6842.679199,2049.941650,37114.316406,2835.542236,1961.512207,3466.286865,1537.509644,0 days 00:33:08.483479971,laser1-LFPo,NREM,Optrode
23710,2022-09-09 12:07:10.483862607,15,18057.483864,0.302608,5963.405273,2227.800781,35636.937500,2763.217041,2048.657471,3206.947998,1485.773315,0 days 00:33:08.483479971,laser1-LFPo,NREM,Optrode


In [194]:
px.box(op_stim.ri().filt_state(), x='channel', y='delta', color='dtype')

In [ ]:
bp['laser1-LFP_']['dtype'] = 'Probe'
bp['laser1-LFPo']['dtype'] = 'Optrode'

In [ ]:
probes = [bp['laser1-LFPo'], bp['laser1-LFP_']]
ps = pd.concat(probes)